# SQL Alchemy Core: Part 1

## Overall Architecture

- The overall architecture is as follows:

**Frontend → API (e.g., Flask or FastAPI) → Backend (e.g., a server running Python) → SQLAlchemy Core (Reads) and SQLAlchemy ORM (Writes) → Database**

- According to [the SQLAlchemy documentation](https://docs.sqlalchemy.org) (2025):
  - **SQLAlchemy Core** is the foundational architecture for SQLAlchemy as a “database toolkit”. The library provides tools for managing connectivity to a database, interacting with database queries and results, and programmatic construction of SQL statements.
  - **SQLAlchemy ORM** builds upon SQLAlchemy Core to provide optional Object Relational Mapping (ORM) capabilities. This includes an additional configuration layer allowing user-defined Python classes to be mapped to database tables and other constructs, as well as an object persistence mechanism known as the Session. It then extends the Core-level SQL Expression Language to allow SQL queries to be composed and invoked in terms of user-defined objects.

## Install SQLAlchemy

- We need to install SQLAlchemy to work with databases in Python.

In [1]:
!pip install sqlalchemy

  Using cached sqlalchemy-2.0.44-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached greenlet-3.3.0-cp313-cp313-win_amd64.whl.metadata (4.2 kB)
Using cached sqlalchemy-2.0.44-cp313-cp313-win_amd64.whl (2.1 MB)
Using cached greenlet-3.3.0-cp313-cp313-win_amd64.whl (301 kB)

   ---------------------------------------- 0/2 [greenlet]
   ---------------------------------------- 0/2 [greenlet]
   ---------------------------------------- 0/2 [greenlet]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlal


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import SQLAlchemy

- We then import the SQLAlchemy library into the Python environment.
- Link to SQLAlchemy documentation: [https://docs.sqlalchemy.org](https://docs.sqlalchemy.org)

In [2]:
import sqlalchemy

The library contains several packages. An alternative way of importing the library is to import specific packages as needed:

In [4]:
from sqlalchemy import create_engine

Another alternative is to import all packages from SQLAlchemy:

In [5]:
from sqlalchemy import *

## Check the Version of SQLAlchemy

- We can confirm the version of SQLAlchemy installed using the `__version__` attribute.

In [3]:
sqlalchemy.__version__

'2.0.44'

## Create an Engine to Connect to the Database

- `create_engine()` is a function in SQLAlchemy that creates a new SQLAlchemy Engine instance. An Engine is the starting point for any SQLAlchemy application that wants to interact with a database. It manages the connection pool and provides a source of database connections.

In [6]:
engine = create_engine('sqlite:///mydatabase.db', echo=True)

- We can then connect to the database using the engine.

In [7]:
conn = engine.connect()

- We can also execute raw SQL statements using the connection object.
- Each SQL statement should be wrapped in the `text()` function from SQLAlchemy.
- The `commit()` method then commits any changes to the database.

In [ ]:
conn.execute(text('CREATE TABLE IF NOT EXISTS people (name VARCHAR(255), age int)'))

conn.commit()

## References
SQLAlchemy Project. (2025, December 5). _SQLAlchemy 2.0 Documentation._ SQLAlchemy. https://docs.sqlalchemy.org/en/20/
